In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Bidirectional,TimeDistributed
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

2025-01-01 21:11:44.459484: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-01 21:11:44.469149: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735787504.478071  178414 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735787504.480713  178414 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-01 21:11:44.490628: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Load DFN and SPMT data with 3.2 V cutoff voltage
# 0.2C
DFN_02C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_02C.csv',header = None))
SPM_02C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_02C.csv',header = None))[:17794]
# 0.5C
DFN_05C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_05C.csv',header = None))[:7011]
SPM_05C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_05C.csv',header = None))
# 1C
DFN_1C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_1C.csv',header = None))[:3426]
SPM_1C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_1C.csv',header = None))
# 2C
DFN_2C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_2C.csv',header = None))[:1648]
SPM_2C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_2C.csv',header = None))
# 3C
DFN_3C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_3C.csv',header = None))[:1067]
SPM_3C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_3C.csv',header = None))
# 4C
DFN_4C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_4C.csv',header = None))[:784]
SPM_4C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_4C.csv',header = None))
# 5C
DFN_5C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_5C.csv',header = None))[:618]
SPM_5C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_5C.csv',header = None))
# 6C
DFN_6C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_6C.csv',header = None))[:509]
SPM_6C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_6C.csv',header = None))
# 7C
DFN_7C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_7C.csv',header = None))[:429]
SPM_7C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_7C.csv',header = None))
# 8C
DFN_8C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_8C.csv',header = None))[:368]
SPM_8C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_8C.csv',header = None))
# 9C
DFN_9C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_9C.csv',header = None))[:321]
SPM_9C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_9C.csv',header = None))
# 10C
DFN_10C = np.array(pd.read_csv('Dataset/SPMTNet/DFN_10C.csv',header = None))[:284]
SPM_10C = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_10C.csv',header = None))
# UDDS
DFN_UDDS = np.array(pd.read_csv('Dataset/SPMTNet/DFN_UDDSx2.csv',header = None))
SPM_UDDS = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_UDDSx2.csv',header = None))[1:]
# US06
DFN_US06 = np.array(pd.read_csv('Dataset/SPMTNet/DFN_US06x6.csv',header = None))
SPM_US06 = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_US06x6.csv',header = None))[1:]
# LA92
DFN_LA92 = np.array(pd.read_csv('Dataset/SPMTNet/DFN_LA92x2.csv',header = None))
SPM_LA92 = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_LA92x2.csv',header = None))[1:]
# SC04
DFN_SC04 = np.array(pd.read_csv('Dataset/SPMTNet/DFN_SC04x4.csv',header = None))
SPM_SC04 = np.array(pd.read_csv('Dataset/SPMTNet/SPMT_SC04x4.csv',header = None))[1:]

In [3]:
# Concatenate 0.2/0.5/1/2/4/6/7/10C and US06/SC04 discharging data for training
# and keep other datasets for testing
DFN = np.concatenate((DFN_02C,DFN_05C),axis=0)
DFN = np.concatenate((DFN,DFN_1C),axis=0)
DFN = np.concatenate((DFN,DFN_2C),axis=0)
# DFN = np.concatenate((DFN,DFN_3C),axis=0)
DFN = np.concatenate((DFN,DFN_4C),axis=0)
# DFN = np.concatenate((DFN,DFN_5C),axis=0)
DFN = np.concatenate((DFN,DFN_6C),axis=0)
# DFN = np.concatenate((DFN,DFN_7C),axis=0)
DFN = np.concatenate((DFN,DFN_8C),axis=0)
# DFN = np.concatenate((DFN,DFN_9C),axis=0)
DFN = np.concatenate((DFN,DFN_10C),axis=0)
# DFN = np.concatenate((DFN,DFN_UDDS),axis=0)
DFN = np.concatenate((DFN,DFN_US06),axis=0)
DFN = np.concatenate((DFN,DFN_LA92),axis=0)
# DFN = np.concatenate((DFN,DFN_SC04),axis=0)

SPM = np.concatenate((SPM_02C,SPM_05C),axis=0)
SPM = np.concatenate((SPM,SPM_1C),axis=0)
SPM = np.concatenate((SPM,SPM_2C),axis=0)
# SPM = np.concatenate((SPM,SPM_3C),axis=0)
SPM = np.concatenate((SPM,SPM_4C),axis=0)
# SPM = np.concatenate((SPM,SPM_5C),axis=0)
SPM = np.concatenate((SPM,SPM_6C),axis=0)
# SPM = np.concatenate((SPM,SPM_7C),axis=0)
SPM = np.concatenate((SPM,SPM_8C),axis=0)
# SPM = np.concatenate((SPM,SPM_9C),axis=0)
SPM = np.concatenate((SPM,SPM_10C),axis=0)
# SPM = np.concatenate((SPM,SPM_UDDS),axis=0)
SPM = np.concatenate((SPM,SPM_US06),axis=0)
SPM = np.concatenate((SPM,SPM_LA92),axis=0)
# SPM = np.concatenate((SPM,SPM_SC04),axis=0)

print("Concatenate SPM train length: " + str(len(SPM)))
print("Concatenate DFN train length: " + str(len(DFN)))

Concatenate SPM train length: 38350
Concatenate DFN train length: 38350


In [4]:
#Construct training data
trainX = np.zeros((len(SPM),4))
# trainX[:,0] = SPM[:,0] # 0: Voltage
trainX[:,0] = SPM[:,1] # 1: Bulk SoC
trainX[:,1] = SPM[:,2] # 2: Surface SoC
trainX[:,2] = SPM[:,3] # 3: Temperature
trainX[:,3] = SPM[:,4] # 4: Current  

# trainY = DFN.reshape(len(DFN),1) - SPM[:,0].reshape(len(SPM),1) # SPMTNet-1: learning voltage residual
trainY = DFN # SPMTNet-2: learning voltage

In [5]:
# normalize input and output data for training
scaler1 = MinMaxScaler()
scaler1.fit(trainX)
normalizedX = scaler1.transform(trainX)
scaler2 = MinMaxScaler()
scaler2.fit(trainY)
normalizedY = scaler2.transform(trainY)
trainX = normalizedX
trainY = normalizedY

In [6]:
initScore = math.sqrt(mean_squared_error(DFN.reshape(len(DFN),1), SPM[:,0].reshape(len(SPM),1)))
print("Initial Score: RMSE " + str(initScore))

Initial Score: RMSE 0.032395216472898364


In [7]:
model = Sequential()
# model.add(SimpleRNN(units=32,input_shape=(look_back + 1,5)))
# # model.add(LSTM(units=32,input_shape=(look_back + 1,4)))
# model.add(Dense(units=1,activation='linear'))

model.add(Dense(32, input_dim=4, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='linear'))

model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

/mnt/c/Users/tuhao/python3_12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1735787518.237630  178414 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9536 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,249 (4.88 KB)

 Trainable params: 1,249 (4.88 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# train NN
model.fit(trainX, trainY, epochs=200, verbose=1)

Epoch 1/200


I0000 00:00:1735787519.455932  178607 service.cc:148] XLA service 0x7f9a34005640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735787519.455954  178607 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2025-01-01 21:11:59.466952: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1735787519.511273  178607 cuda_dnn.cc:529] Loaded cuDNN version 90300


 216/1199 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - loss: 0.0268

I0000 00:00:1735787520.413441  178607 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1199/1199 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0085 
Epoch 2/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 1s 663us/step - loss: 2.9223e-04
Epoch 3/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 1s 682us/step - loss: 1.8709e-04
Epoch 4/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 1s 666us/step - loss: 1.4521e-04
Epoch 5/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - loss: 1.2057e-04
Epoch 6/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 1s 811us/step - loss: 1.0228e-04
Epoch 7/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 9.4209e-05
Epoch 8/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 8.4681e-05
Epoch 9/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 8.2632e-05
Epoch 10/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.0212e-05
Epoch 11/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 1s 874us/step - loss: 7.2904e-05
Epoch 12/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.9453e-05
Epoch 13/200
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 1s 905us/step - loss: 6.4987e-05
Epoch 14/200
1199/1199 ━━━━━━━━━━━━

In [9]:
trainPredict = model.predict(trainX)
trainPredict = scaler2.inverse_transform(trainPredict.reshape(len(trainPredict),1))
trainScore = math.sqrt(mean_squared_error(DFN, trainPredict.reshape(len(trainPredict),1)))
print("Train Score: RMSE " + str(trainScore))

1199/1199 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Train Score: RMSE 0.005288601710673378


In [10]:
# Testing data separately
SPM_T = SPM_UDDS
DFN_T = DFN_UDDS

#Construct testing data
testX = np.zeros((len(SPM_T),4))
testX[:,0] = SPM_T[:,1]
testX[:,1] = SPM_T[:,2]
testX[:,2] = SPM_T[:,3]
testX[:,3] = SPM_T[:,4]
# testX[:,4] = SPM_T[:,5]

# # SPMTNet-1: learning voltage residual
# testY =  DFN_T.reshape(len(DFN_T),1) - SPM_T[:,0].reshape(len(SPM_T),1) 
# SPMTNet-2: learning voltage
testY = DFN_T

In [11]:
# Prediction of the testing case
normalizedXtest = scaler1.transform(testX)
testPredict = model.predict(normalizedXtest)
testPredict = scaler2.inverse_transform(testPredict)
testScore = math.sqrt(mean_squared_error(testY,testPredict.reshape(len(testPredict),1)))
print("Test Score: RMSE " + str(testScore))
InitialScore = math.sqrt(mean_squared_error(DFN_T, SPM_T[:,0]))
print("Initial Score: RMSE " + str(InitialScore))

109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Test Score: RMSE 0.010212729545114731
Initial Score: RMSE 0.02768061551249933


In [ ]:
# Save data for the hybrid model
np.savetxt('Journal_results/HYBRID2/predict_Test_UDDS.csv', testPredict.reshape(len(testPredict),1), delimiter=',')